# Hospital Readmission Risk Model - Data Exploration

**Project:** Hospital Readmission Risk Prediction  
**Timeline:** January 2015 - May 2015  
**Author:** Blake Sonnier  
**Team:** Data Science Intern, Mentor, Clinical Advisor  

## Objective
Explore raw EHR data from Southeast Texas regional hospitals to understand:
- Data quality issues and inconsistencies
- Patient demographics and clinical patterns
- Readmission patterns and potential risk factors
- Data preprocessing requirements

**Note:** This notebook demonstrates the analysis process using synthetic data that mimics the real EHR data patterns encountered during the internship.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Import our custom modules
import sys
sys.path.append('../src')
from data_processing import EHRDataProcessor
from visualization import ReadmissionVisualizer

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully")
print(f"Analysis conducted on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 1. Data Loading and Initial Assessment

In [ ]:
# Load the raw EHR dataset
raw_data = pd.read_csv('../data/sample_data.csv')
print(f"Dataset loaded: {raw_data.shape}")

# Initial data inspection
print("=== INITIAL DATA OVERVIEW ===")
print(raw_data.head())
print("\n=== DATA TYPES ===")
print(raw_data.dtypes)
print("\n=== BASIC STATISTICS ===")
print(raw_data.describe())

## 2. Data Quality Assessment

### Key Finding: Significant data quality issues requiring preprocessing

In [ ]:
# Initialize data processor for quality assessment
processor = EHRDataProcessor()
quality_report = processor.validate_data_quality(raw_data)

print("=== DATA QUALITY REPORT ===")
print(f"Total records: {quality_report['total_records']:,}")
print(f"Total columns: {quality_report['total_columns']}")
print(f"Duplicates: {quality_report['duplicates']}")
print(f"Issues found: {len(quality_report['issues'])}")

# Missing values analysis
print("\n=== MISSING VALUES ANALYSIS ===")
missing_stats = pd.DataFrame({
    'Column': raw_data.columns,
    'Missing_Count': raw_data.isnull().sum(),
    'Missing_Percentage': (raw_data.isnull().sum() / len(raw_data)) * 100
})
missing_stats = missing_stats.sort_values('Missing_Percentage', ascending=False)
print(missing_stats[missing_stats['Missing_Count'] > 0])

In [ ]:
# Visualize data quality using our custom visualizer
visualizer = ReadmissionVisualizer()
visualizer.plot_data_quality_overview(raw_data)

# Analyze inconsistent formatting issues
print("=== DATA INCONSISTENCY ANALYSIS ===")

# Gender field inconsistencies
print("\nGender field variations:")
print(raw_data['gender'].value_counts())

# Insurance type inconsistencies
print("\nInsurance type variations:")
print(raw_data['insurance_type'].value_counts())

# Date format issues
print("\nSample admission dates (showing format inconsistencies):")
print(raw_data['admission_date'].head(10).tolist())

# Diagnosis codes complexity
print("\nSample diagnosis codes (showing mixed coding systems):")
print(raw_data['diagnosis_codes'].head(10).tolist())

## 3. Exploratory Data Analysis

### Patient Demographics and Clinical Patterns

In [ ]:
# Clinical overview visualization
visualizer.plot_clinical_overview(raw_data)

# Key statistics
print(f"\n=== KEY STATISTICS ===")
print(f"Average age: {raw_data['age'].mean():.1f} years")
print(f"Average length of stay: {raw_data['length_of_stay'].mean():.1f} days")
print(f"30-day readmission rate: {(raw_data['readmission_30_day'].sum() / len(raw_data)) * 100:.1f}%")
print(f"Emergency admission rate: {(raw_data['emergency_admission'].sum() / len(raw_data)) * 100:.1f}%")

## 4. Risk Factor Analysis

### Identifying patterns associated with readmission risk

In [ ]:
# Readmission risk by age groups
raw_data['age_group'] = pd.cut(raw_data['age'], 
                              bins=[0, 40, 60, 80, 100], 
                              labels=['Under 40', '40-60', '60-80', 'Over 80'])

risk_by_age = raw_data.groupby('age_group')['readmission_30_day'].agg(['mean', 'count']).reset_index()
risk_by_age['readmission_rate'] = risk_by_age['mean'] * 100

plt.figure(figsize=(15, 8))

plt.subplot(2, 3, 1)
plt.bar(risk_by_age['age_group'].astype(str), risk_by_age['readmission_rate'])
plt.xlabel('Age Group')
plt.ylabel('Readmission Rate (%)')
plt.title('Readmission Rate by Age Group')
plt.xticks(rotation=45)

# Risk by length of stay
plt.subplot(2, 3, 2)
raw_data['los_group'] = pd.cut(raw_data['length_of_stay'], 
                              bins=[0, 2, 5, 10, 100], 
                              labels=['1-2 days', '3-5 days', '6-10 days', '>10 days'])

risk_by_los = raw_data.groupby('los_group')['readmission_30_day'].mean() * 100
plt.bar(risk_by_los.index.astype(str), risk_by_los.values)
plt.xlabel('Length of Stay')
plt.ylabel('Readmission Rate (%)')
plt.title('Readmission Rate by Length of Stay')
plt.xticks(rotation=45)

# Risk by previous admissions
plt.subplot(2, 3, 3)
risk_by_prev = raw_data.groupby('previous_admissions')['readmission_30_day'].mean() * 100
risk_by_prev = risk_by_prev.head(6)  # Show first 6 categories
plt.bar(risk_by_prev.index.astype(str), risk_by_prev.values)
plt.xlabel('Previous Admissions')
plt.ylabel('Readmission Rate (%)')
plt.title('Readmission Rate by Previous Admissions')

# Risk by emergency admission
plt.subplot(2, 3, 4)
risk_by_emergency = raw_data.groupby('emergency_admission')['readmission_30_day'].mean() * 100
emergency_labels = ['Scheduled', 'Emergency']
plt.bar(emergency_labels, risk_by_emergency.values)
plt.xlabel('Admission Type')
plt.ylabel('Readmission Rate (%)')
plt.title('Readmission Rate by Admission Type')

# Correlation heatmap for numeric variables
plt.subplot(2, 3, 5)
numeric_cols = ['age', 'length_of_stay', 'previous_admissions', 'emergency_admission', 'readmission_30_day']
correlation_matrix = raw_data[numeric_cols].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, square=True)
plt.title('Feature Correlation Matrix')

# Distribution of LOS for readmitted vs non-readmitted
plt.subplot(2, 3, 6)
readmitted = raw_data[raw_data['readmission_30_day'] == 1]['length_of_stay']
not_readmitted = raw_data[raw_data['readmission_30_day'] == 0]['length_of_stay']

plt.hist(not_readmitted, bins=20, alpha=0.7, label='Not Readmitted', density=True)
plt.hist(readmitted, bins=20, alpha=0.7, label='Readmitted', density=True)
plt.xlabel('Length of Stay')
plt.ylabel('Density')
plt.title('LOS Distribution by Readmission Status')
plt.legend()

plt.tight_layout()
plt.show()

## 5. Medical Codes Analysis

### Challenge: Multiple coding systems and translations

In [ ]:
# Analyze diagnosis codes patterns
print("=== MEDICAL CODES ANALYSIS ===")

# Extract all unique codes
all_codes = []
for codes_string in raw_data['diagnosis_codes']:
    if pd.notna(codes_string) and codes_string != '':
        codes = codes_string.split(';')
        all_codes.extend([code.strip() for code in codes if code.strip()])

code_counts = pd.Series(all_codes).value_counts()
print(f"\nTotal unique codes found: {len(code_counts)}")
print(f"Most common codes:")
print(code_counts.head(15))

# Identify code types
icd9_pattern = code_counts[code_counts.index.str.contains(r'\d{3}\.\d', na=False)]
icd10_pattern = code_counts[code_counts.index.str.contains(r'[A-Z]\d{2}\.', na=False)]
local_codes = code_counts[~code_counts.index.str.contains(r'\d{3}\.\d|[A-Z]\d{2}\.', na=False)]

print(f"\nCode system breakdown:")
print(f"ICD-9 like codes: {len(icd9_pattern)}")
print(f"ICD-10 like codes: {len(icd10_pattern)}")
print(f"Local/Other codes: {len(local_codes)}")

# Visualize code distribution
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
code_types = ['ICD-9', 'ICD-10', 'Local/Other']
code_type_counts = [len(icd9_pattern), len(icd10_pattern), len(local_codes)]
plt.pie(code_type_counts, labels=code_types, autopct='%1.1f%%')
plt.title('Distribution of Code Types')

plt.subplot(1, 2, 2)
top_codes = code_counts.head(10)
plt.barh(range(len(top_codes)), top_codes.values)
plt.yticks(range(len(top_codes)), top_codes.index)
plt.xlabel('Frequency')
plt.title('Top 10 Most Common Codes')
plt.gca().invert_yaxis()

plt.tight_layout()
plt.show()

## 6. Data Quality Issues Summary

### Key challenges identified for preprocessing pipeline

In [ ]:
# Comprehensive data quality report
print("=== DATA QUALITY ISSUES SUMMARY ===")
print("\n1. MISSING VALUES:")
for col in missing_stats[missing_stats['Missing_Percentage'] > 0]['Column']:
    pct = missing_stats[missing_stats['Column'] == col]['Missing_Percentage'].iloc[0]
    print(f"   - {col}: {pct:.1f}% missing")

print("\n2. DATA INCONSISTENCIES:")
print(f"   - Gender: {len(raw_data['gender'].unique())} different formats")
print(f"   - Insurance: {len(raw_data['insurance_type'].unique())} different formats")
print(f"   - Date formats: Multiple inconsistent formats detected")
print(f"   - Medical codes: {len(code_counts)} unique codes across 3+ systems")

print("\n3. DATA OUTLIERS:")
outlier_los = raw_data[raw_data['length_of_stay'] > 20]
print(f"   - Length of stay outliers: {len(outlier_los)} patients (>20 days)")

extreme_age = raw_data[(raw_data['age'] < 18) | (raw_data['age'] > 100)]
print(f"   - Age outliers: {len(extreme_age)} patients (<18 or >100 years)")

print("\n4. PREPROCESSING REQUIREMENTS:")
print("   ✓ Standardize gender and insurance fields")
print("   ✓ Parse and normalize date formats")
print("   ✓ Translate medical codes to common standard")
print("   ✓ Handle missing values appropriately")
print("   ✓ Detect and treat outliers")
print("   ✓ Create time-series features from admission history")

print("\n=== BUSINESS INSIGHTS ===")
print(f"✓ {raw_data['readmission_30_day'].mean()*100:.1f}% baseline readmission rate")
print(f"✓ Emergency admissions have higher readmission risk")
print(f"✓ Previous admission history is strong predictor")
print(f"✓ Length of stay shows correlation with readmission")
print(f"✓ Age groups show varying risk patterns")

## 7. Next Steps and Recommendations

Based on this exploratory analysis, the following preprocessing steps are critical:

### Immediate Actions Required:
1. **Data Standardization Pipeline**: Build robust parsers for inconsistent formats
2. **Medical Code Translation**: Create mapping tables for ICD-9, ICD-10, and local codes
3. **Missing Value Strategy**: Develop domain-appropriate imputation methods
4. **Feature Engineering**: Extract meaningful patterns from temporal data

### Clinical Collaboration Points:
- **Code Mapping Validation**: Work with clinical advisor to ensure accurate translations
- **Feature Relevance**: Validate that engineered features align with clinical understanding
- **Risk Thresholds**: Establish clinically meaningful risk categories

### Technical Implementation:
- Pandas-based standardization functions
- Robust date parsing with multiple format support
- Time-series feature extraction for admission patterns
- Quality control checks and validation rules

**This analysis revealed the complexity of real-world EHR data and the critical importance of thorough data exploration before model development. The insights gained here directly informed our feature engineering and preprocessing strategies.**

In [ ]:
# Save data quality report for next notebook
data_quality_report = {
    'total_patients': len(raw_data),
    'readmission_rate': raw_data['readmission_30_day'].mean(),
    'missing_value_columns': missing_stats[missing_stats['Missing_Percentage'] > 0]['Column'].tolist(),
    'unique_codes_count': len(code_counts),
    'preprocessing_required': True
}

print("\n=== EXPLORATION COMPLETE ===")
print("Data quality report prepared for feature engineering phase.")
print("\nKey files to create next:")
print("- 02_feature_engineering.ipynb")
print("- Medical code mapping tables")
print("- Data standardization functions")

# Preview of cleaned data structure for next phase
print("\n=== PREVIEW: Target Clean Data Structure ===")
print("Columns after preprocessing:")
target_columns = [
    'patient_id', 'age', 'gender_std', 'admission_date_parsed',
    'days_since_last_admission', 'diagnosis_diabetes', 'diagnosis_hypertension',
    'diagnosis_heart_disease', 'diagnosis_kidney_disease', 'length_of_stay',
    'previous_admissions_imputed', 'emergency_admission', 'insurance_std',
    'readmission_30_day'
]

for i, col in enumerate(target_columns, 1):
    print(f"{i:2d}. {col}")